# Crawling 실습
## Selenium을 사용한 동적 웹 크롤링
### 아카이브 사이트 (https://arxiv.org/) 크롤링

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# 검색어 입력받기
search_word = input("검색어를 입력해 주세요 : ")

검색어를 입력해 주세요 : data science


In [3]:
# 크롬 드라이버 불러오기
driver = webdriver.Chrome('D:/chromedriver_win32/chromedriver.exe') 
driver.implicitly_wait(3)

In [4]:
driver.get('https://arxiv.org/') # 크롤링하고자 하는 사이트 주소 입력

search = driver.find_element_by_xpath('//*[@id="header"]/div[2]/form/div/div[1]/input') # 검색 창 xpath 
search.clear()
search.send_keys(search_word) # 입력 받은 검색어 검색창에 입력
time.sleep(1)
search.send_keys(Keys.ENTER) # 엔터 키 누름

**페이지 당 나타나는 게시물의 수: 50개**

In [5]:
# 리스트 생성
total_title = [] # 제목
total_author = [] # 저자
total_date = [] # 제출일자

**제목 바뀌는 형태**
- #main-container > div.content > ol > li:nth-child(1) > p.title.is-5.mathjax  
- #main-container > div.content > ol > li:nth-child(2) > p.title.is-5.mathjax

**저자 바뀌는 형태**
- #main-container > div.content > ol > li:nth-child(1) > p.authors  
- #main-container > div.content > ol > li:nth-child(2) > p.authors  
'Authors: 저자1, 저자2, ...' 형태로 출력 -> 'Authors:' 삭제

**제출일자 바뀌는 형태**  
- #main-container > div.content > ol > li:nth-child(1) > p:nth-child(5)
- #main-container > div.content > ol > li:nth-child(2) > p:nth-child(5)  
'Submitted 날짜; originally announced 날짜' 형태로 출력 -> 첫번째 날짜만 가져옴

In [6]:
for i in range(1, 51):  ## 1페이지에 게시물 50개씩 존재

    # 제목
    title = driver.find_element_by_css_selector(
        f"#main-container > div.content > ol > li:nth-child({i}) > p.title.is-5.mathjax")
    time.sleep(1)
    total_title.append(title.text)
    
    # 저자
    author = driver.find_element_by_css_selector(
        f"#main-container > div.content > ol > li:nth-child({i}) > p.authors")
    time.sleep(1)
    new_author = author.text.replace('Authors: ','')
    total_author.append(new_author)
    
    # 제출일자
    date = driver.find_element_by_css_selector(
        f'#main-container > div.content > ol > li:nth-child({i}) > p:nth-child(5)')
    time.sleep(1)
    sub_date = date.text.split(';')[0]
    new_sub_date = sub_date.replace('Submitted ','')
    total_date.append(new_sub_date)

In [8]:
# 세 개의 리스트 데이터 프레임 형태로 저장
data = {"제목": total_title, "저자" : total_author, "제출일자" : total_date}
df = pd.DataFrame(data, columns = ["제목", "저자", "제출일자"])
df

,제목,저자,제출일자
0,Discovering the Information that is lost in ou...,"P. D. Bruza, H. A. Proper","18 May, 2021"
1,Learning and Certification under Instance-targ...,"Ji Gao, Amin Karbasi, Mohammad Mahmoody","18 May, 2021"
2,Durable Queues: The Second Amendment,"Gal Sela, Erez Petrank","18 May, 2021"
3,Content Disentanglement for Semantically Consi...,"Mert Keser, Artem Savkin, Federico Tombari","18 May, 2021"
4,Learning Embeddings from Knowledge Graphs With...,"Sumit Pai, Luca Costabello","18 May, 2021"
5,The Computational Complexity of ReLU Network T...,"Vincent Froese, Christoph Hertrich, Rolf Niede...","18 May, 2021"
6,DID-eFed: Facilitating Federated Learning as a...,"Jiahui Geng, Neel Kanwal, Martin Gilje Jaatun,...","18 May, 2021"
7,Deep Graph Convolutional Reinforcement Learnin...,"Farzan Soleymani, Eric Paquet","6 May, 2021"
8,DCAP: Deep Cross Attentional Product Network f...,"Zekai Chen, Fangtian Zhong, Zhumin Chen, Xiao ...","18 May, 2021"
9,rta-dq-lib: a software library to perform onli...,"Leonardo Baroncelli, Andrea Bulgarelli, Nicolo...","18 May, 2021"
